# Sowing Success: How Machine Learning Helps Farmers Select the Best Crops

![Farmer in a field](farmer_in_a_field.jpg)

Measuring essential soil metrics such as nitrogen, phosphorous, potassium levels, and pH value is an important aspect of assessing soil condition. However, it can be an expensive and time-consuming process, which can cause farmers to prioritize which metrics to measure based on their budget constraints.

Farmers have various options when it comes to deciding which crop to plant each season. Their primary objective is to maximize the yield of their crops, taking into account different factors. One crucial factor that affects crop growth is the condition of the soil in the field, which can be assessed by measuring basic elements such as nitrogen and potassium levels. Each crop has an ideal soil condition that ensures optimal growth and maximum yield.

A farmer reached out to you as a machine learning expert for assistance in selecting the best crop for his field. They've provided you with a dataset called `soil_measures.csv`, which contains:

- `"N"`: Nitrogen content ratio in the soil
- `"P"`: Phosphorous content ratio in the soil
- `"K"`: Potassium content ratio in the soil
- `"pH"` value of the soil
- `"crop"`: categorical values that contain various crops (target variable).

Each row in this dataset represents various measures of the soil in a particular field. Based on these measurements, the crop specified in the `"crop"` column is the optimal choice for that field.  

In this project, you will build multi-class classification models to predict the type of `"crop"` and identify the single most importance feature for predictive performance.

In [163]:
# All required libraries are imported here for you.
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

# Load the dataset
crops = pd.read_csv("soil_measures.csv")

# Write your code here
print(crops.head())
print(crops.shape)

    N   P   K        ph  crop
0  90  42  43  6.502985  rice
1  85  58  41  7.038096  rice
2  60  55  44  7.840207  rice
3  74  35  40  6.980401  rice
4  78  42  42  7.628473  rice
(2200, 5)


In [164]:
#Step 1 : EDA

print("Crop types (Outcome possibilities) and class balance")
print(crops.crop.value_counts())
print(crops.crop.unique())
print(crops.crop.unique().size)

print("\nCheck for missing data")
print(crops.isna().sum())

Crop types (Outcome possibilities) and class balance
rice           100
maize          100
jute           100
cotton         100
coconut        100
papaya         100
orange         100
apple          100
muskmelon      100
watermelon     100
grapes         100
mango          100
banana         100
pomegranate    100
lentil         100
blackgram      100
mungbean       100
mothbeans      100
pigeonpeas     100
kidneybeans    100
chickpea       100
coffee         100
Name: crop, dtype: int64
['rice' 'maize' 'chickpea' 'kidneybeans' 'pigeonpeas' 'mothbeans'
 'mungbean' 'blackgram' 'lentil' 'pomegranate' 'banana' 'mango' 'grapes'
 'watermelon' 'muskmelon' 'apple' 'orange' 'papaya' 'coconut' 'cotton'
 'jute' 'coffee']
22

Check for missing data
N       0
P       0
K       0
ph      0
crop    0
dtype: int64


In [165]:
#Step 2 : Split test and train data

y = crops['crop'].values.reshape(-1, 1)
X = crops.drop(columns='crop', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

#each test_train split for each feature in a dict
#each feature (key) has a list value with index 0 as train data and index 1 as test data
features_train_test = {}

for col in X_train.columns:
    features_train_test[col] = [list(X_train[col].values.reshape(-1, 1)), list(X_test[col].values.reshape(-1, 1))]
    
print(y_train.shape, y_test.shape)

(1980, 1) (220, 1)


In [166]:
#Step 3 : Make and train model for each feature 

#dict with feature (key) and model (value) 
models = {}
#models instantiation and training
for col in X_train.columns:
    models[col] = LogisticRegression().fit(features_train_test[col][0], y_train)

In [167]:
#Step 4 : Test and evaluate each feature performance

features_accuracies = {}
best_predictive_feature = {}
highest_acc = 0

#test
for col in X_train.columns:
    y_pred = models[col].predict(features_train_test[col][1])
    features_accuracies[col] = metrics.accuracy_score(y_test, y_pred)
    if (highest_acc < features_accuracies[col]) :
        highest_acc = features_accuracies[col]
        
#print accuracy of model with each feature
print(features_accuracies)

for feature, acc in features_accuracies.items():
    if (acc == highest_acc):
        best_predictive_feature[feature] = acc
        
#print best feature with respective model accuracy
print(best_predictive_feature)

{'N': 0.15454545454545454, 'P': 0.21363636363636362, 'K': 0.3090909090909091, 'ph': 0.09090909090909091}
{'K': 0.3090909090909091}


In [ ]:
# All required libraries are imported here for you.
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

# Load the dataset
crops = pd.read_csv("soil_measures.csv")

# Check for missing values
crops.isna().sum()

# Check how many crops we have, i.e., multi-class target
crops.crop.unique()

# Split into feature and target sets
X = crops.drop(columns="crop")
y = crops["crop"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

# Create a dictionary to store the model performance for each feature
feature_performance = {}

# Train a logistic regression model for each feature
for feature in ["N", "P", "K", "ph"]:
    log_reg = LogisticRegression(multi_class="multinomial")
    log_reg.fit(X_train[[feature]], y_train)
    y_pred = log_reg.predict(X_test[[feature]])
    
    # Calculate F1 score, the harmonic mean of precision and recall
    # Could also use balanced_accuracy_score
    f1 = metrics.f1_score(y_test, y_pred, average="weighted")
    
    # Add feature-f1 score pairs to the dictionary
    feature_performance[feature] = f1
    print(f"F1-score for {feature}: {f1}")

# K produced the best F1 score
# Store in best_predictive_feature dictionary
best_predictive_feature = {"K": feature_performance["K"]}
best_predictive_feature